Integrantes:
    Daniel Ramos.
    Joaquín Fernando Aguilar.
    Davis Guerra.

In [1]:
import pyspark
sc = pyspark.SparkContext('local[*]')

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

## Leer los datos

In [2]:
df = sqlContext.read.format("csv").option("delimiter", "\t").option("header", "true").load("cubo_ventas/fact_item.txt")

In [3]:
df_clientes = sqlContext.read.format("csv").option("delimiter", "\t").option("header", "true").load("cubo_ventas/dim_customer.txt")

In [4]:
df.head()

Row(dwh_country_id='276', fk_sales_order_item='88056', fk_sales_order='5642', local_unit_price='1.29', unit_price_eur='1.29', local_tax_amount='0.08', tax_amount_eur='0.08', local_paid_price='1.29', paid_price_eur='1.29', local_paid_price_net='1.21', paid_price_net_eur='1.21', local_coupon_money_value='0', coupon_money_value_eur='0', local_coupon_money_value_net='0', coupon_money_value_eur_net='0', local_original_price='1.29', original_unit_price_eur='0.89', fk_dim_catalog='1676', fk_dim_date='9/28/2015 12:00:00 AM', fk_dim_customer='194', fk_dim_order='6142', order_bi_created_at='9/28/2015 8:33:17 AM', order_bi_updated_at='1/26/2016 3:55:59 PM', order_item_bi_created_at='9/28/2015 8:33:17 AM', order_item_bi_updated_at='9/28/2015 8:34:04 AM')

In [5]:
df.printSchema()

root
 |-- dwh_country_id: string (nullable = true)
 |-- fk_sales_order_item: string (nullable = true)
 |-- fk_sales_order: string (nullable = true)
 |-- local_unit_price: string (nullable = true)
 |-- unit_price_eur: string (nullable = true)
 |-- local_tax_amount: string (nullable = true)
 |-- tax_amount_eur: string (nullable = true)
 |-- local_paid_price: string (nullable = true)
 |-- paid_price_eur: string (nullable = true)
 |-- local_paid_price_net: string (nullable = true)
 |-- paid_price_net_eur: string (nullable = true)
 |-- local_coupon_money_value: string (nullable = true)
 |-- coupon_money_value_eur: string (nullable = true)
 |-- local_coupon_money_value_net: string (nullable = true)
 |-- coupon_money_value_eur_net: string (nullable = true)
 |-- local_original_price: string (nullable = true)
 |-- original_unit_price_eur: string (nullable = true)
 |-- fk_dim_catalog: string (nullable = true)
 |-- fk_dim_date: string (nullable = true)
 |-- fk_dim_customer: string (nullable = tru

In [24]:
df_clientes.printSchema()

root
 |-- id_dim_customer: string (nullable = true)
 |-- dwh_country_id: string (nullable = true)
 |-- region_code: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- first_order_id: string (nullable = true)
 |-- first_order_date: string (nullable = true)
 |-- unit_price_total_eur: string (nullable = true)
 |-- paid_price_total_eur: string (nullable = true)
 |-- coupon_value_total_eur: string (nullable = true)
 |-- coupon_value_total_eur_net: string (nullable = true)
 |-- tax_total_eur: string (nullable = true)
 |-- cogs_sum_eur: string (nullable = true)
 |-- shipping_amount_eur: string (nullable = true)
 |-- net_revenue_eur: string (nullable = true)
 |-- orders: string (nullable = true)
 |-- is_employee: string (nullable = true)
 |-- order_nr: string (nullable = true)
 |-- channel: string (nullable = true)



In [6]:
df.describe('dwh_country_id').show()

+-------+--------------+
|summary|dwh_country_id|
+-------+--------------+
|  count|        175909|
|   mean|         276.0|
| stddev|           0.0|
|    min|           276|
|    max|           276|
+-------+--------------+



## SQL API

In [7]:
df.createOrReplaceTempView("fact_item")
df_clientes.createOrReplaceTempView("dim_customer")
users = sqlContext.sql("SELECT channel,fk_sales_order_item \
                       FROM fact_item join dim_customer on fk_dim_customer=id_dim_customer limit 10")
users.show()

+-----------------+-------------------+
|          channel|fk_sales_order_item|
+-----------------+-------------------+
|Social Media Paid|              88056|
|Social Media Paid|              88055|
|Social Media Paid|              88054|
|Social Media Paid|              88053|
|Social Media Paid|              88052|
|Social Media Paid|              88051|
|Social Media Paid|              91950|
|Social Media Paid|              91949|
|Social Media Paid|              91948|
|Social Media Paid|              91947|
+-----------------+-------------------+



### 1. Perfilamiento de datos

Realice un perfilamiento de datos para 4 columnas de fact_item que hagan referencia a una métrica. 

Las columnas seleccionadas son unit_price_eur, tax_amount_eur, paid_price_eur y fk_dim_order.
Primero hago una descripción general de las columnas.
La media tiene sentido porque el impuesto pagado siempre es menor al precio original del producto, y el precio pagado ya tiene aplicados descuentos, por lo cual se podría decir que las medias están relativamente bien.
De las filas min y max se puede observar que los precios de los productos oscilan entre un centavo y 10 euros, y el impuesto sí empieza desde cero porque hay productos no gravados.

In [8]:
df.describe('unit_price_eur','tax_amount_eur','paid_price_eur','fk_dim_order').show()

+-------+------------------+-------------------+-----------------+------------------+
|summary|    unit_price_eur|     tax_amount_eur|   paid_price_eur|      fk_dim_order|
+-------+------------------+-------------------+-----------------+------------------+
|  count|            175909|             175909|           175909|            175909|
|   mean|3.2467645202904185|0.21976345724218566|2.732469856573652|  5500.85149139612|
| stddev| 4.929455440993037| 0.4750312949782069|4.437943426162388|3373.9870972214635|
|    min|               0.1|                  0|                0|                 1|
|    max|              9.99|               9.58|            97.64|              9999|
+-------+------------------+-------------------+-----------------+------------------+



In [ ]:
Primero evaluamos la cantidad de productos por cada orden y el valor total pagado en cada una de ellas.

In [14]:
ordenes = sqlContext.sql("SELECT fk_dim_order AS numorden, \
                       COUNT(*) AS numprods, \
                       ROUND(SUM(paid_price_eur)) AS totalorden \
                       FROM fact_item GROUP BY fk_dim_order")
ordenes.show()

+--------+--------+----------+
|numorden|numprods|totalorden|
+--------+--------+----------+
|    9586|      24|      88.0|
|    7711|      21|      31.0|
|    7273|      48|     141.0|
|    5645|      26|     102.0|
|    8304|      17|      33.0|
|    9993|      15|      98.0|
|    6613|      11|      42.0|
|    4937|      18|      40.0|
|    6194|      15|      45.0|
|    9583|      25|      51.0|
|    5325|      32|      91.0|
|     467|      20|      90.0|
|    7762|       8|      45.0|
|    9009|       6|      50.0|
|     691|       8|      32.0|
|   14887|       1|      26.0|
|   15271|       1|      43.0|
|     675|      11|      58.0|
|    3959|      21|      54.0|
|    9030|      19|      66.0|
+--------+--------+----------+
only showing top 20 rows



In [ ]:
Ahora se analizan las columnas con valores nulos o diferentes a caracteres numéricos para el caso de los precios,
impuestos y total.

In [15]:
nulos = sqlContext.sql("SELECT COUNT(*) FROM fact_item WHERE unit_price_eur IS NULL")
nulos.show()

+--------+
|count(1)|
+--------+
|       0|
+--------+



In [21]:
letras = sqlContext.sql("SELECT COUNT(*) FROM fact_item WHERE unit_price_eur NOT REGEXP '^[0-9]+(\.[0-9]+)?$'")
letras.show()

+--------------+
|unit_price_eur|
+--------------+
+--------------+



In [16]:
nulos = sqlContext.sql("SELECT COUNT(*) FROM fact_item WHERE tax_amount_eur IS NULL")
nulos.show()

+--------+
|count(1)|
+--------+
|       0|
+--------+



In [22]:
letras = sqlContext.sql("SELECT COUNT(*) FROM fact_item WHERE tax_amount_eur NOT REGEXP '^[0-9]+(\.[0-9]+)?$'")
letras.show()

+--------+
|count(1)|
+--------+
|       0|
+--------+



In [17]:
nulos = sqlContext.sql("SELECT COUNT(*) FROM fact_item WHERE paid_price_eur IS NULL")
nulos.show()

+--------+
|count(1)|
+--------+
|       0|
+--------+



In [23]:
letras = sqlContext.sql("SELECT COUNT(*) FROM fact_item WHERE paid_price_eur NOT REGEXP '^[0-9]+(\.[0-9]+)?$'")
letras.show()

+--------+
|count(1)|
+--------+
|       0|
+--------+



Entonces, de acuerdo a las consultas anteriores, ninguno de los campos validados tiene valores nulos o no numéricos en sus registros, por lo cual se podrían hacer cálculos con ellos sin incurrir en errores.

### 2. Ejercicio SQL:

a.Ventas (fact_item.paid_price_total_eur) por canal de mercadeo (dim_custome.channel)


Se sustituye la columna paid_price_total_eur por paid_price_net_eur porque la primera no existe en la tabla.

In [29]:
ventas = sqlContext.sql("SELECT d.channel AS canal, ROUND(SUM(f.paid_price_net_eur)) AS ventas \
    FROM fact_item f JOIN dim_customer d ON f.fk_dim_customer = d.id_dim_customer \
    GROUP BY d.channel \
    ORDER BY ventas DESC")
ventas.show()

+--------------------+-------+
|               canal| ventas|
+--------------------+-------+
|              Direct|90341.0|
|                 SEO|70365.0|
|           SEM Brand|49080.0|
|                null|34338.0|
|            Referral|30122.0|
|       SEM Non Brand|13150.0|
|   Social Media Paid|12012.0|
|        Other Unpaid| 7855.0|
| Social Media Unpaid| 7766.0|
|               Email| 5848.0|
|             Display| 1888.0|
|Social Media Reta...| 1768.0|
|         Retargeting| 1730.0|
+--------------------+-------+



b.Número de clientes por método de pago (dim_order.payment_method)

In [ ]:
df_orden = sqlContext.read.format("csv").option("delimiter", "\t").option("header", "true").load("cubo_ventas/dim_order.txt")
df_orden.createOrReplaceTempView("dim_order")

In [32]:
clientes = sqlContext.sql("SELECT o.payment_method AS metodo, COUNT(DISTINCT(f.fk_dim_customer)) AS clientes \
    FROM fact_item f \
    JOIN dim_order o ON f.fk_dim_order = o.id_dim_order \
    GROUP BY o.payment_method")
clientes.show()

+--------------------+--------+
|              metodo|clientes|
+--------------------+--------+
|           NoPayment|     169|
|                 COD|     559|
|    Adyen_CreditCard|    1727|
|   Adyen_DirectDebit|     853|
|Adyen_PaypalRecur...|       7|
|        Adyen_Paypal|    2407|
+--------------------+--------+



c.Ranking de productores (dim_catalog.producer) por ventas (fact_item.paid_price_total_eur)

In [33]:
df_catalogo = sqlContext.read.format("csv").option("delimiter", "\t").option("header", "true").load("cubo_ventas/dim_catalog.txt")
df_catalogo.createOrReplaceTempView("dim_catalog")

In [34]:
df_catalogo.printSchema()

root
 |-- id_catalog: string (nullable = true)
 |-- dwh_country_id: string (nullable = true)
 |-- fk_catalog_config: string (nullable = true)
 |-- config_sku: string (nullable = true)
 |-- fk_catalog_simple: string (nullable = true)
 |-- simple_sku: string (nullable = true)
 |-- name: string (nullable = true)
 |-- description: string (nullable = true)
 |-- producer_id: string (nullable = true)
 |-- producer: string (nullable = true)
 |-- fk_catalog_brand: string (nullable = true)



In [35]:
productores = sqlContext.sql("SELECT p.name, t.ventas \
    FROM dim_catalog p JOIN ( \
      SELECT f.fk_dim_catalog, ROUND(SUM(f.paid_price_net_eur)) AS ventas \
      FROM fact_item f \
      GROUP BY f.fk_dim_catalog \
    ) t ON t.fk_dim_catalog = p.id_catalog \
    ORDER BY t.ventas DESC")
productores.show()

+--------------------+------+
|                name|ventas|
+--------------------+------+
|Office Box Obst &...|6698.0|
|       Regionale Box|6224.0|
|     Die Grüne Kiste|5839.0|
|            Obst Box|5705.0|
|10 Bio Eier aus W...|4946.0|
|          Gemüse Box|4256.0|
|Wöchentliche Smoo...|4121.0|
|Wöchentliche Smoo...|3921.0|
|Der Bonativo Adve...|3288.0|
|Gemüse Wocheneink...|3046.0|
|Loué Hähnchenbrus...|2834.0|
|         Fitness Box|2667.0|
|Rinderfilet in Sc...|2254.0|
|Bio Kartoffeln Li...|2163.0|
|       Hähnchenkeule|2030.0|
|      Berliner Biere|2012.0|
|             Avocado|1956.0|
|6 Bio Eier aus Wi...|1922.0|
|Bergkäse - jung (...|1831.0|
|             Bananen|1777.0|
+--------------------+------+
only showing top 20 rows



d. Ranking de clientes por ventas (fact_item.paid_price_total_eur

In [38]:
rankingclientes = sqlContext.sql("SELECT f.fk_dim_customer, ROUND(SUM(f.paid_price_net_eur)) AS ventas \
    FROM fact_item f \
    GROUP BY f.fk_dim_customer \
    ORDER BY ventas DESC")
rankingclientes.show()

+---------------+------+
|fk_dim_customer|ventas|
+---------------+------+
|           4477|4053.0|
|           1265|3821.0|
|           5416|3700.0|
|           3654|3082.0|
|           2750|3012.0|
|           5144|2856.0|
|           3806|2698.0|
|            535|2363.0|
|            586|2090.0|
|           5064|2034.0|
|           3407|1914.0|
|           2751|1890.0|
|           1175|1827.0|
|           3918|1813.0|
|           1964|1663.0|
|           3815|1632.0|
|           5776|1574.0|
|           2196|1563.0|
|           3445|1524.0|
|            946|1498.0|
+---------------+------+
only showing top 20 rows



e. Número de órdenes con más de 3 items y con valor pagado por item mayor a 30 euros (fact_item.paid_price_total_eur)

In [49]:
ordenesgrandes = sqlContext.sql("SELECT COUNT(t.*) AS ordenes \
                       FROM ( \
                         SELECT fk_dim_order AS idorden, COUNT(*) AS items \
                         FROM fact_item \
                         WHERE paid_price_net_eur > 30 \
                         GROUP BY fk_dim_order \
                         HAVING items > 3 \
                       ) t")
ordenesgrandes.show()

+-------+
|ordenes|
+-------+
|      7|
+-------+



Ahora se ejecuta el conteo con spark.

In [52]:
ordenesgrandes = sqlContext.sql("SELECT fk_dim_order AS idorden, COUNT(*) AS items \
                         FROM fact_item \
                         WHERE paid_price_net_eur > 30 \
                         GROUP BY fk_dim_order \
                         HAVING items > 3")
ordenesgrandes.count()

7

### Ejercicios PySpark

Elija dos de las preguntas del punto anterior y respóndalas utilizando las funciones de PySpark
http://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html
https://www.analyticsvidhya.com/blog/2016/10/spark-dataframe-and-operations/
